In [1]:
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import glob
import io

In [2]:
import neuralcoref
import spacy

In [3]:
print('spacy version:', spacy.__version__)
print('neuralcoref version:', neuralcoref.__version__)

spacy version: 2.1.3
neuralcoref version: 4.0.0


1. id_file
2. extract_ner_par id_file(book)
    a) book = standardize_text(text))
        text = retrieve_text(id_file) # open file
    b) paragraphs, chapters = remove_chapter_markers(book, id_file)

In [6]:
id_file = "ZaneGrey_TheManoftheForest"

In [102]:
# a)
# retrieve_text

f = io.open(id_file + '.txt', 'r', encoding='utf-8')
text = f.read()
f.close()
print(text)

﻿The Project Gutenberg EBook of The Man of the Forest, by Zane Grey

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: The Man of the Forest

Author: Zane Grey

Posting Date: February 12, 2009 [EBook #3457]
Last Updated: March 10, 2018

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK THE MAN OF THE FOREST ***




Produced by Richard Fane





THE MAN OF THE FOREST

by Zane Grey


Harper and Brothers

New York

1920

Published: 1919




CHAPTER I

At sunset hour the forest was still, lonely, sweet with tang of fir and
spruce, blazing in gold and red and green; and the man who glided on
under the great trees seemed to blend with the colors and, disappearing,
to have become a part of the wild woodland.

Old Baldy, highest of the White Mounta

In [101]:
keywords = ["*** START OF THIS PROJECT GUTENBERG"]

start_index = None
for keyword in keywords:
    start_match = re.search(re.escape(keyword), text)
    if start_match:
        start_index = start_match.end()
        break

main_content = text[start_index:]

print(main_content)

 EBOOK THE MAN OF THE FOREST ***




Produced by Richard Fane





THE MAN OF THE FOREST

by Zane Grey


Harper and Brothers

New York

1920

Published: 1919




CHAPTER I

At sunset hour the forest was still, lonely, sweet with tang of fir and
spruce, blazing in gold and red and green; and the man who glided on
under the great trees seemed to blend with the colors and, disappearing,
to have become a part of the wild woodland.

Old Baldy, highest of the White Mountains, stood up round and bare,
rimmed bright gold in the last glow of the setting sun. Then, as the
fire dropped behind the domed peak, a change, a cold and darkening
blight, passed down the black spear-pointed slopes over all that
mountain world.

It was a wild, richly timbered, and abundantly watered region of dark
forests and grassy parks, ten thousand feet above sea-level, isolated
on all sides by the southern Arizona desert--the virgin home of elk and
deer, of bear and lion, of wolf and fox, and the birthplace as well as

In [14]:
# a) 
# standardize text

book = text.replace('\r', '')
book = book.replace('\n\n', '\r')
book = book.replace('\r', '\n\n')
book = book.replace('..', '.')
book = book.replace('..', '...')
book = book.replace('_', '')
book = book.replace('-', ' ')
book

"\ufeffThe Project Gutenberg EBook of The Man of the Forest, by Zane Grey\n\nThis eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org\n\n\nTitle: The Man of the Forest\n\nAuthor: Zane Grey\n\nPosting Date: February 12, 2009 [EBook #3457]\nLast Updated: March 10, 2018\n\nLanguage: English\n\nCharacter set encoding: UTF 8\n\n*** START OF THIS PROJECT GUTENBERG EBOOK THE MAN OF THE FOREST ***\n\n\n\n\nProduced by Richard Fane\n\n\n\n\n\nTHE MAN OF THE FOREST\n\nby Zane Grey\n\n\nHarper and Brothers\n\nNew York\n\n1920\n\nPublished: 1919\n\n\n\n\nCHAPTER I\n\nAt sunset hour the forest was still, lonely, sweet with tang of fir and\nspruce, blazing in gold and red and green; and the man who glided on\nunder the great trees seemed to blend with the colors and, disappearing,\nto have become a part of t

In [16]:
# b) 
# paragraphs, chapters = remove_chapter_markers(book, id_file)

paragraphs = book.split('\n\n')
paragraphs

['\ufeffThe Project Gutenberg EBook of The Man of the Forest, by Zane Grey',
 'This eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org',
 '\nTitle: The Man of the Forest',
 'Author: Zane Grey',
 'Posting Date: February 12, 2009 [EBook #3457]\nLast Updated: March 10, 2018',
 'Language: English',
 'Character set encoding: UTF 8',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE MAN OF THE FOREST ***',
 '',
 '\nProduced by Richard Fane',
 '',
 '',
 'THE MAN OF THE FOREST',
 'by Zane Grey',
 '\nHarper and Brothers',
 'New York',
 '1920',
 'Published: 1919',
 '',
 '\nCHAPTER I',
 'At sunset hour the forest was still, lonely, sweet with tang of fir and\nspruce, blazing in gold and red and green; and the man who glided on\nunder the great trees seemed to blend with the colors and, disappearing,\nto ha

In [79]:
paragraphs_new = []
contents = []
chapters = []
old_idx = 0

for paragraph in paragraphs:
    # print("\n Paragraph: ", paragraph)
    
    lines = paragraph.split('\n')
    # print(lines)
    if re.match('[IVXLCDM\d]+[\.]*', lines[0].upper().strip()):
        if len(contens) == 0:
            contents = [" ".join(line.lower().split()) for line in lines]
            print("contents", contents)
            
            if len(contents) == 0:
                print("Not found content", id_file)
                exit()
        continue
           
    if paragraph.lower().strip() in contents:
        if old_idx == 0:
            old_idx = len(paragraphs_new)
        else:
            current_idx = len(paragraphs_new)
            chapter = "\n\n".join(paragraphs_new[old_idx:current_idx])
            chapters.append(chapter)
            old_idx = current_idx
        continue
    
    if not re.match('.*[\w]+.*', paragraph):
        continue
    
    paragraphs_new.append(paragraph)
 

    if old_idx != 0:
        chapter = "\n\n".join(paragraphs_new[old_idx:len(paragraphs_new)])
        chapters.append(chapter)
        
print(len(paragraphs), len(paragraphs_new),
      #len(chapters)
     )    

3808 3491


In [80]:
paragraphs_new

['\ufeffThe Project Gutenberg EBook of The Man of the Forest, by Zane Grey',
 'This eBook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  You may copy it, give it away or\nre use it under the terms of the Project Gutenberg License included\nwith this eBook or online at www.gutenberg.org',
 'Author: Zane Grey',
 'Posting Date: February 12, 2009 [EBook #3457]\nLast Updated: March 10, 2018',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE MAN OF THE FOREST ***',
 'THE MAN OF THE FOREST',
 'by Zane Grey',
 'New York',
 'Published: 1919',
 'At sunset hour the forest was still, lonely, sweet with tang of fir and\nspruce, blazing in gold and red and green; and the man who glided on\nunder the great trees seemed to blend with the colors and, disappearing,\nto have become a part of the wild woodland.',
 'Old Baldy, highest of the White Mountains, stood up round and bare,\nrimmed bright gold in the last glow of the setting sun. Then, as the\nfire droppe

In [81]:
# extract_ner_par id_file
book_x = book
paragraphs_x = paragraphs_new

In [88]:
nlp = spacy.load('en_core_web_sm')
neuralcoref.add_to_pipe(nlp)

In [99]:
geral_entities = []

for i, pa in enumerate(paragraphs_x[0:15]):
    pa = pa.replace('\n', ' ')
    pa = " ".join(pa.split())
    print('\n', pa)
    
    doc = nlp(pa)
    print(doc)
    
    doc_coref = doc._.coref_resolved
    doc_coref = nlp(doc_coref)
    print(doc_coref)
    
    entities = []
    for ent in doc_coref.ents:
        if ent.label_ == "PERSON":
            entities.append(ent.text)
    # print(entities)
    
    if len(entities) > 0:
        len_words = len(pa.split())
        # print(len_words)
        geral_entities.extend(entities)
        print(len_words, entities)
    
print(set(geral_entities))
    
    


 ﻿The Project Gutenberg EBook of The Man of the Forest, by Zane Grey
﻿The Project Gutenberg EBook of The Man of the Forest, by Zane Grey
﻿The Project Gutenberg EBook of The Man of the Forest, by Zane Grey
13 ['Zane Grey']

 This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org
This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org
This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy This eBook, give This eBook away or re use This eBook under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org
45 ['Gu